In [1]:
!pip install psycopg2-binary
!pip install boto3
!pip install python-dotenv
!pip install pandas
!pip install SQLAlchemy

In [2]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import psycopg2

load_dotenv()

False

### Uncomment the below code cell if you decide to set and load the credentials from an environment variable. You'll however need to create a `.env` file in the working directory

In [3]:
# driver=os.environ.get('POSTGRES_DRIVER')
# host=os.environ.get('POSTGRES_HOST')
# port=os.environ.get('POSTGRES_PORT')
# database=os.environ.get('POSTGRES_DATABASE')
# username=os.environ.get('POSTGRES_USERNAME')
# password=os.environ.get('POSTGRES_PASSWORD')
# staging_schema=os.environ.get('POSTGRES_STAGING_SCHEMA')
# analytics_schema=os.environ.get('POSTGRES_ANALYTICS_SCHEMA')

In [4]:
driver='postgresql+psycopg2'
host='34.89.230.185'
port=5432
database='d2b_accessment'
username='ifeaakaw4441'
password='OcCwxlTINF'
staging_schema='ifeaakaw4441_staging'
analytics_schema='ifeaakaw4441_analytics'
bucket_name='d2b-internal-assessment-bucket'

In [5]:
# create engine that connects to the database
engine = create_engine(f'{driver}://{username}:{password}@{host}:{port}/{database}')
engine.connect()
print('Engine connected successfully!')

Engine connected successfully!


In [6]:
# create cursor object
connection = psycopg2.connect(database=database,
                        host=host,
                        user=username,
                        password=password,
                        port=port
                             )
connection.autocommit = True
cursor = connection.cursor()
print('Cursor object created')

Cursor object created


In [7]:
# Download all the files using boto3
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = "d2b-internal-assessment-bucket"
response = s3.list_objects(Bucket=bucket_name, Prefix="orders_data")
files_to_download = ['orders.csv','reviews.csv','shipment_deliveries.csv']

for file in files_to_download:
    s3.download_file(bucket_name, f"orders_data/{file}", f"{file}")

### Read the downloaded files into pandas Dataframes

In [8]:
orders_df = pd.read_csv('orders.csv')
reviews_df = pd.read_csv('reviews.csv')
shipment_deliveries_df = pd.read_csv('shipment_deliveries.csv')

### Check

In [9]:
orders_df.head()

,order_id,customer_id,order_date,product_id,unit_price,quantity,total_price
0,1,5,2022-07-13,24,139,10,1390
1,2,14,2021-04-06,2,273,4,1092
2,3,17,2022-07-29,20,253,9,2277
3,4,14,2022-08-27,8,334,1,334
4,5,25,2021-12-15,6,334,3,1002


In [10]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_id     10000 non-null  int64 
 1   customer_id  10000 non-null  int64 
 2   order_date   10000 non-null  object
 3   product_id   10000 non-null  int64 
 4   unit_price   10000 non-null  int64 
 5   quantity     10000 non-null  int64 
 6   total_price  10000 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 547.0+ KB


In [11]:
reviews_df.head()

,review,product_id
0,1,21
1,3,1
2,2,8
3,1,5
4,5,22


In [12]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7236 entries, 0 to 7235
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   review      7236 non-null   int64
 1   product_id  7236 non-null   int64
dtypes: int64(2)
memory usage: 113.2 KB


In [13]:
shipment_deliveries_df.head()

,shipment_id,order_id,shipment_date,delivery_date
0,1,1,2022-07-14,NaN
1,2,2,NaN,NaN
2,3,3,2022-07-31,2022-08-03
3,4,4,2022-09-02,2022-09-05
4,5,5,2021-12-19,2021-12-20


In [14]:
shipment_deliveries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   shipment_id    10000 non-null  int64 
 1   order_id       10000 non-null  int64 
 2   shipment_date  3243 non-null   object
 3   delivery_date  2197 non-null   object
dtypes: int64(2), object(2)
memory usage: 312.6+ KB


### Write Raw data to database using pandas

In [15]:
# write orders raw data to staging database
orders_df.to_sql(name='orders', 
                 schema=staging_schema, 
                 con=engine, 
                 if_exists='replace', 
                 index=False) 
print(f"Loaded raw orders data into {staging_schema} successfully!")

# write reviews raw data to staging database
reviews_df.to_sql(name='reviews', 
                 schema=staging_schema, 
                 con=engine, 
                 if_exists='replace', 
                 index=False) 
print(f"Loaded raw reviews data into {staging_schema} successfully!")

# write shipment_deliveries raw data to staging database
shipment_deliveries_df.to_sql(name='shipment_deliveries', 
                 schema=staging_schema, 
                 con=engine, 
                 if_exists='replace', 
                 index=False) 
print(f"Loaded raw shipment_deliveries data into {staging_schema} successfully!")

Loaded raw orders data into ifeaakaw4441_staging successfully!
Loaded raw reviews data into ifeaakaw4441_staging successfully!
Loaded raw shipment_deliveries data into ifeaakaw4441_staging successfully!


### Execute SQL Scripts

In [16]:
def read_sql_scripts(file_path):
    with open(file_path, 'r') as file_handle:
        cursor.execute(file_handle.read())
        print(f"Performed Transformations and created {file_path.split('/')[-1].split('.')[0]} table in {analytics_schema}")

In [17]:
path = './sql_scripts'  
for file in os.listdir(path):
    if file.endswith(".sql"):
        full_file_path = f"{path}/{file}"
        read_sql_scripts(full_file_path)

Performed Transformations and created best_performing_product table in ifeaakaw4441_analytics
Performed Transformations and created agg_public_holiday table in ifeaakaw4441_analytics
Performed Transformations and created agg_shipments table in ifeaakaw4441_analytics


### Load the Transformed Tables to s3

In [18]:
# download the transformed tables
transformed_path = './transformed'
transformed_tables_names = ['best_performing_product', 'agg_public_holiday', 'agg_shipments']
for table in transformed_tables_names:
    df_transformed = pd.read_sql_query(f'select * from {analytics_schema}.{table}', con=engine)
    df_transformed.to_csv(f'{transformed_path}/{table}.csv', index=False)

In [19]:
# Upload to s3
for file in os.listdir(transformed_path):
    if file.endswith(".csv"):
        full_file_path = f"{transformed_path}/{file}"
        s3.upload_file(full_file_path, bucket_name, f'analytics_export/ifeaakaw4441/{file}')

In [22]:
my_submission_check = s3.list_objects(Bucket=bucket_name, Prefix="analytics_export/ifeaakaw4441/")
my_submission_check

{'ResponseMetadata': {'RequestId': '8EVBQ9TPRJWD10NB',
  'HostId': '7NyOsRXFitKtuodxZtjBEC0MmmxU++8Qvi433y84W/Fu2qKWF0+CVFfvUdUpCaFtohE+XFRVzZ3hyR67jfdvaw==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7NyOsRXFitKtuodxZtjBEC0MmmxU++8Qvi433y84W/Fu2qKWF0+CVFfvUdUpCaFtohE+XFRVzZ3hyR67jfdvaw==',
   'x-amz-request-id': '8EVBQ9TPRJWD10NB',
   'date': 'Sat, 24 Jun 2023 22:05:14 GMT',
   'x-amz-bucket-region': 'eu-central-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'analytics_export/ifeaakaw4441/agg_public_holiday.csv',
   'LastModified': datetime.datetime(2023, 6, 24, 22, 2, 36, tzinfo=tzutc()),
   'ETag': '"951d934c168726c6e90cb0f586e27f5d"',
   'Size': 260,
   'StorageClass': 'STANDARD',
   'Owner': {'ID': '65a011a29cdf8ec533ec3d1ccaae921c'}},
  {'Key': 'analytics_export/ifeaakaw4441/agg_shipments.csv',
   'LastModified': datetime.datet